In [1]:
%run data_preprocess.ipynb
%run models.ipynb

In [2]:
import torch.optim as optim

In [3]:
encoder = encoderRNN(4096, 512, 0.3)
decoder = decoderRNN()
model = MODELS(encoder=encoder, decoder=decoder)

In [4]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
def calculate_loss(x, y, lenghts):
    batch_size = len(x)
    predict_cat = None
    groundT_cat = None
    flag = True

    for batch in range(batch_size):
        predict = x[batch]
        ground_truth = y[batch]
        seq_len = lengths[batch] -1

        predict = predict[:seq_len]
        ground_truth = ground_truth[:seq_len]
        if flag:
            predict_cat = predict
            groundT_cat = ground_truth
            flag = False
        else:
            predict_cat = torch.cat((predict_cat, predict), dim=0)
            groundT_cat = torch.cat((groundT_cat, ground_truth), dim=0)

    try:
        assert len(predict_cat) == len(groundT_cat)

    except AssertionError as error:
        print('prediction length is not same as ground truth length')
        print('prediction length: {}, ground truth length: {}'.format(len(predict_cat), len(groundT_cat)))

    loss = loss_fn(predict_cat, groundT_cat)
    avg_loss = loss/batch_size

    return loss

In [6]:
def train(model, epoch):    
    for batch_idx, batch in enumerate(train_dataloader):
        avi_feat, ground_truth, length = batch
        avi_feat, ground_truth = avi_feat.cuda(), ground_truth.cuda()
        avi_feat, ground_truth = Variable(avi_feat), Variable(ground_truth)
        optimizer.zero_grad()
        predicted, seq_predictions = model(avi_feat, target_sentences=ground_truth, mode='train', tr_steps=epoch)
        ground_truth = ground_truth[:, 1:]
        loss = calculate_loss(predicted, ground_truth, lengths)
        loss.backward()
        optimizer.step()
    loss = loss.item()

In [8]:
epochs_n = 5
model = model.cuda()
for epoch in range(epochs_n):
    train(model, epoch+1)

AttributeError: 'tuple' object has no attribute 'size'